In [3]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
from scipy.stats import ttest_ind

# Load the datasets into DataFrames
df = pd.read_csv('stand-alone-policy-or-plan-for-mental-health.csv')
source1_df = pd.read_csv('death-rates-from-mental-and-substance-disorders-by-age-who.csv')


# Merge the DataFrames based on both 'Country' and 'Year' columns
merged_df = pd.merge(df, source1_df, on=['Entity', 'Year'], how='inner')


# Save the filtered data to a new file
df_final = merged_df[merged_df["Year"] == 2014]
df_final.to_csv('merged_data.csv', index=False)

In [4]:
# Separate the data into two groups
insurance_yes = df_final[df_final["Indicator:Stand-alone policy or plan for mental health"] == "Yes"]
insurance_no = df_final[df_final["Indicator:Stand-alone policy or plan for mental health"] == "No"]


In [5]:
# Descriptive statistics
desc_yes = insurance_yes[["Death rate per 100,000 people - Cause: Mental and substance use disorders - Sex: Both sexes - Age_group: ALLAges"]].describe()
desc_no = insurance_no[["Death rate per 100,000 people - Cause: Mental and substance use disorders - Sex: Both sexes - Age_group: ALLAges"]].describe()

print("With Health Insurance:")
print(desc_yes)
print("\nWithout Health Insurance:")
print(desc_no)


With Health Insurance:
       Death rate per 100,000 people - Cause: Mental and substance use disorders - Sex: Both sexes - Age_group: ALLAges
count                                         128.000000                                                               
mean                                            3.565937                                                               
std                                             3.993127                                                               
min                                             0.340000                                                               
25%                                             1.290000                                                               
50%                                             2.270000                                                               
75%                                             3.485000                                                               
max              

In [6]:
# Perform t-test for death rate
insurance_yes_2016 = insurance_yes[["Death rate per 100,000 people - Cause: Mental and substance use disorders - Sex: Both sexes - Age_group: ALLAges"]]
insurance_no_2016 = insurance_no[["Death rate per 100,000 people - Cause: Mental and substance use disorders - Sex: Both sexes - Age_group: ALLAges"]]

t_stat_death_rate, p_value_death_rate = ttest_ind(insurance_yes_2016, insurance_no_2016, equal_var=False)

print(f"T-test for Death Rate: t-statistic = {t_stat_death_rate}, p-value = {p_value_death_rate}")

# H0 is null hypothesis that there is no difference
# Hypothesis testing
alpha = 0.05
if p_value_death_rate < alpha:
    print("Reject the null hypothesis: There is a significant difference.")
else:
    print("Fail to reject the null hypothesis: There is no significant difference.")

T-test for Death Rate: t-statistic = [-1.55737975], p-value = [0.12736745]
Fail to reject the null hypothesis: There is no significant difference.


In [7]:
# Perform bootstrapping
def bootstrap(data1, data2, num_samples=1000):
    means_diff = []
    for _ in range(num_samples):
        sample1 = data1.sample(frac=1, replace=True)
        sample2 = data2.sample(frac=1, replace=True)
        mean_diff = sample1.mean() - sample2.mean()
        means_diff.append(mean_diff)
    return np.array(means_diff)

np.random.seed(0)  # For reproducibility
bootstrap_death_rate = bootstrap(insurance_yes_2016, insurance_no_2016)
conf_interval_death_rate = np.percentile(bootstrap_death_rate, [2.5, 97.5])


print(f"95% Confidence Interval for Death Rate: {conf_interval_death_rate}")

# Check if zero is within the confidence interval
significant_death_rate = not (conf_interval_death_rate[0] <= 0 <= conf_interval_death_rate[1])

print(f"Significant difference in Death Rate: {significant_death_rate}")



95% Confidence Interval for Death Rate: [-5.05786602  0.15906205]
Significant difference in Death Rate: False
